<a href="https://colab.research.google.com/github/SiwawawaXD/LLMclassification/blob/main/pythonCallChatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q openai

In [4]:
import numpy as np

x = np.linspace(-5, 5, 15)   # 15 points between -5 and 5
y = x**2
points = np.column_stack((x, y))  # shape (15,2)
print("Parabola points:\n", points)

Parabola points:
 [[-5.         25.        ]
 [-4.28571429 18.36734694]
 [-3.57142857 12.75510204]
 [-2.85714286  8.16326531]
 [-2.14285714  4.59183673]
 [-1.42857143  2.04081633]
 [-0.71428571  0.51020408]
 [ 0.          0.        ]
 [ 0.71428571  0.51020408]
 [ 1.42857143  2.04081633]
 [ 2.14285714  4.59183673]
 [ 2.85714286  8.16326531]
 [ 3.57142857 12.75510204]
 [ 4.28571429 18.36734694]
 [ 5.         25.        ]]


In [7]:
import os
from openai import OpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize client with your API key from Colab userdata
client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY")
)

# Send request using chat.completions
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": "You are a math assistant."},
        {"role": "user", "content": f"Here are 15 points from a curve: {points.tolist()}. "
                                    "Based on these points, what general shape is this curve?"},
    ],#temperature=0,
    max_completion_tokens=1024
    #max_tokens=1024
)

# Print the assistant's reply
print(response.choices[0].message.content)


Parabola (specifically y = x^2). The points lie on a symmetric, U-shaped curve with vertex at (0,0) and y ≈ x^2 for all x.
